In [ ]:
import pandas as pd
import json
import numpy as np
import requests
from config import (client_id, api_key)
from pprint import pprint
import pydbgen
from pydbgen import pydbgen
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
business_data={"Business ID":[],
               "Business Name":[],
               "Business Alias":[],
               "Category":[],
               "City":[],
               "State":[],
               "ZipCode":[],
               "Lat":[],
               "Long":[],
               "Review Count":[],
               "Rating":[],
              }
review_data={"Business ID":[],
             "Review":[],
            }

In [ ]:
myDB=pydbgen.pydb()
city= []
lat = []
long = []
for x in range(600):
    x=(myDB.city_real())
    #print (x)
    if x not in city:
        city.append(x)
city_yelp=pd.DataFrame(city)
city_yelp=city_yelp.rename(columns={0:"US Cities"})

In [ ]:
api_key=api_key
headers = {"Authorization": (f"Bearer {api_key}")}
url="https://api.yelp.com/v3/businesses/search?"
review_url="https://api.yelp.com/v3/businesses/"
categories="lawyers"
radius = 40000
counter = 1

In [ ]:
for index, row in city_yelp.iterrows():
    try:
        random_city = row['US Cities']
        print(f"{counter}: {random_city}")
        yelp_response2 = requests.get((f"{url}categories={categories}&location={random_city}&radius={radius}"), headers=headers, verify=False).json()
        #pprint(yelp_response2)
        counter+=1
    except:
        print(f"{counter}: Could not locate any lawyers in {random_city}.")
        #print(yelp_response2)
    #print(len(yelp_response2["businesses"]))
    try:    
        for x in range(len(yelp_response2["businesses"])):
            try:
                business_data["Business ID"].append(yelp_response2["businesses"][x]["id"])
                business_data["Business Name"].append(yelp_response2["businesses"][x]["name"])
                business_data["Business Alias"].append(yelp_response2["businesses"][x]["alias"])
                business_data["Category"].append(yelp_response2["businesses"][x]["categories"][0]["alias"])
                business_data["City"].append(yelp_response2["businesses"][x]["location"]["city"])
                business_data["State"].append(yelp_response2["businesses"][x]["location"]["state"])
                business_data["ZipCode"].append(yelp_response2["businesses"][x]["location"]["zip_code"])
                business_data["Lat"].append(yelp_response2["businesses"][x]["coordinates"]["latitude"])
                business_data["Long"].append(yelp_response2["businesses"][x]["coordinates"]["longitude"])
                business_data["Review Count"].append(yelp_response2["businesses"][x]["review_count"])
                business_data["Rating"].append(yelp_response2["businesses"][x]["rating"])
            except: 
                print(f"     No law offices found in {random_city} using Yelp.com.")
    except:
        print(f"     No law offices found in {random_city} using Yelp.com.")
        counter+=1

In [ ]:
graph_data = pd.DataFrame({ key:pd.Series(value) for key, value in business_data.items() })

In [ ]:
for id in business_data["Business ID"]:
    try:
        yelp_response3 = requests.get((f"{review_url}{id}/reviews"), headers=headers, verify=False).json()
        pprint(yelp_response3)
        print(f"Fetching Reviews for {id}.")
    except:
        print(f"No Reviews for {id}.")
    try:
        for x in range(len(yelp_response3["reviews"])):
            try:
                review_data["Business ID"].append(id)
                review_data["Review"].append(yelp_response3["reviews"][x]["text"])
            except:
                print("skipping")
    except:
        print("skipping")

In [ ]:
graph_review = pd.DataFrame({ key:pd.Series(value) for key, value in review_data.items() })

In [ ]:
graph_review["Compound"]=""
graph_review.head()

for index, row in graph_review.iterrows():
    results = analyzer.polarity_scores(row["Review"])
    graph_review.loc[index, "Compound"] = results["compound"]

In [ ]:
final=graph_review.merge(graph_data, on="Business ID", how="left")

In [ ]:
final.to_csv(path_or_buf="Merged6.csv", sep=',')